# Prepare environment

In [1]:
!pip install git+https://github.com/katarinagresova/ensembl_scraper.git

  Cloning https://github.com/katarinagresova/ensembl_scraper.git to /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-flj6kfiz
  Running command git clone --filter=blob:none -q https://github.com/katarinagresova/ensembl_scraper.git /private/var/folders/q7/pcbd1fnn22l233m3cmvvx1nm0000gn/T/pip-req-build-flj6kfiz
  Resolved https://github.com/katarinagresova/ensembl_scraper.git to commit 2c4b3a55383851f9fdaa3ca7470f69b3489a468f
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for scraper, since package 'wheel' is not installed.
  Attempting uninstall: scraper
    Found existing installation: scraper 0.0.1
    Uninstalling scraper-0.0.1:
      Successfully uninstalled scraper-0.0.1
    Running setup.py install for scraper ... done


# Create config file

In [2]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "homo_sapiens": {
            "external_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

# Prepare directories

In [6]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/human_tss_ensembl/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

# Run tool

In [7]:
!python -m scraper.ensembl_scraper -c user_config.yaml

Processing feature files:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:download_file(): Going to download file from path ftp://ftp.ensembl.org/pub/release-100/mysql/regulation_mart_100/hsapiens_external_feature__external_feature__main.txt.gz
INFO:root:download_file(): File ../../datasets//tmp//homo_sapiens_external_feature.txt.gz already exists. Not going to download.
INFO:root:parse_feature_file(): Going to parse file ../../datasets//tmp//homo_sapiens_external_feature.txt.gz
INFO:root:parse_feature_file(): Done parsing file ../../datasets//tmp//homo_sapiens_external_feature.txt.gz


Processing feature types:   0%|                           | 0/2 [00:00<?, ?it/s]INFO:root:find_sequences(): Going to find sequences based on genomic loci.
INFO:root:download_2bit_file(): Going to download 2bit file hg38
INFO:root:download_2bit_file(): File for hg38 already exists. Not going to download.
INFO:root:find_sequences(): Done finding sequences.
INFO:root:remove_low_quality(): G

# Cleaning

In [8]:
!mv ../../datasets/homo_sapiens_external_feature_transcription_start_site ../../datasets/human_tss_ensembl/

In [9]:
!rm requirements.txt user_config.yaml

rm: requirements.txt: No such file or directory


In [10]:
!rm -rf ../../datasets/tmp/

In [12]:
!rm -rf ../../datasets/homo_sapiens_external_feature_enhancer

# Final reformating

  * gzip all CSV files
  * add extra formatting to yaml config file

In [13]:
!find ../../datasets/human_tss_ensembl/ -type f -name "*.csv" -exec gzip {} \;

In [14]:
with open("../../datasets/human_tss_ensembl/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'}},
 'version': 0}

In [15]:
config['classes']['positive']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 
config['classes']['negative']['extra_processing'] = 'ENSEMBL_HUMAN_GENOME' 

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'}},
 'version': 0}

In [ ]:
with open("../../datasets/human_tss_ensembl/metadata.yaml", 'w') as handle:
  yaml.dump(config, handle)